In [1]:
pip install numpy matplotlib opencv-python


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

In [3]:
def translation_matrix(t):
    return np.array([
        [1, 0, 0, t[0]],
        [0, 1, 0, t[1]],
        [0, 0, 1, t[2]],
        [0, 0, 0, 1]
    ])

def scaling_matrix(s):
    return np.array([
        [s[0], 0, 0, 0],
        [0, s[1], 0, 0],
        [0, 0, s[2], 0],
        [0, 0, 0, 1]
    ])

def rotation_matrix_x(theta):
    return np.array([
        [1, 0, 0, 0],
        [0, np.cos(theta), -np.sin(theta), 0],
        [0, np.sin(theta), np.cos(theta), 0],
        [0, 0, 0, 1]
    ])

def rotation_matrix_y(theta):
    return np.array([
        [np.cos(theta), 0, np.sin(theta), 0],
        [0, 1, 0, 0],
        [-np.sin(theta), 0, np.cos(theta), 0],
        [0, 0, 0, 1]
    ])

def rotation_matrix_z(theta):
    return np.array([
        [np.cos(theta), -np.sin(theta), 0, 0],
        [np.sin(theta), np.cos(theta), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

def combine_transformations(transformations):
    M = np.eye(4)
    for transform in transformations:
        M = transform @ M
    return M

def apply_transformation(M, points):
    points = np.hstack([points, np.ones((points.shape[0], 1))])
    transformed_points = (M @ points.T).T
    return transformed_points[:, :3]

In [4]:
def generate_frames(output_dir='frames', num_frames=60):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Define a simple 3D object (cube)
    cube = np.array([
        [1, 1, 1],
        [1, -1, 1],
        [-1, -1, 1],
        [-1, 1, 1],
        [1, 1, -1],
        [1, -1, -1],
        [-1, -1, -1],
        [-1, 1, -1]
    ])
    
    edges = [
        [0, 1], [1, 2], [2, 3], [3, 0],
        [4, 5], [5, 6], [6, 7], [7, 4],
        [0, 4], [1, 5], [2, 6], [3, 7]
    ]
    
    for i in range(num_frames):
        theta = (i / num_frames) * 2 * np.pi
        scaling = scaling_matrix([1 + i / num_frames, 1 + ((i*4) / num_frames), 1 + i / num_frames])
        rotation = rotation_matrix_z(theta)
        
        # Combine transformations
        M = combine_transformations([scaling, rotation])
        transformed_cube = apply_transformation(M, cube)
        
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        
        for edge in edges:
            ax.plot(
                [transformed_cube[edge[0], 0], transformed_cube[edge[1], 0]],
                [transformed_cube[edge[0], 1], transformed_cube[edge[1], 1]],
                [transformed_cube[edge[0], 2], transformed_cube[edge[1], 2]], 'b'
            )
        
        ax.set_xlim([-3, 3])
        ax.set_ylim([-3, 3])
        ax.set_zlim([-3, 3])
        plt.axis('off')
        
        frame_path = os.path.join(output_dir, f'frame_{i:03d}.png')
        plt.savefig(frame_path)
        plt.close()

generate_frames()

In [5]:
def create_video(output_dir='frames', video_filename='transformation_video.mp4', fps=30):
    frame_files = sorted([f for f in os.listdir(output_dir) if f.endswith('.png')])
    frame = cv2.imread(os.path.join(output_dir, frame_files[0]))
    height, width, layers = frame.shape

    video = cv2.VideoWriter(video_filename, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    for frame_file in frame_files:
        frame = cv2.imread(os.path.join(output_dir, frame_file))
        video.write(frame)

    cv2.destroyAllWindows()
    video.release()

create_video()

In [6]:
from IPython.display import Video

Video('transformation_video.mp4')

In [7]:
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def generate_frames(output_dir='frames', num_frames=60):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Define the origin
    origin = np.array([0, 0, 0])

    # Define 25 vectors of different lengths between 1 and 2 units
    vectors = np.random.randn(25, 3)  # Randomly generate 25 direction vectors

    # Normalize each vector to make them between 1 and 2 units in length
    lengths = 1 + np.random.rand(25)  # Generate random lengths between 1 and 2
    normalized_vectors = vectors / np.linalg.norm(vectors, axis=1)[:, np.newaxis] * lengths[:, np.newaxis]
    
    # Generate a list of random colors for each vector
    colors = np.random.rand(25, 3)  # 25 colors in RGB format

    for i in range(num_frames):
        theta = (i / num_frames) * 2 * np.pi
        scaling = scaling_matrix([1 + i / num_frames, 1 + ((i * 2) / num_frames), 1 + ((i * .25) / num_frames)])
        rotation = rotation_matrix_z(theta)
        
        # Combine transformations
        M = combine_transformations([scaling, rotation])
        transformed_vectors = apply_transformation(M, normalized_vectors)
        
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        
        for vector, color in zip(transformed_vectors, colors):
            ax.quiver(
                origin[0], origin[1], origin[2],
                vector[0], vector[1], vector[2],
                length=np.linalg.norm(vector), normalize=True, color=color,
                arrow_length_ratio=0.1  # Remove the arrowheads
            )
        
        ax.set_xlim([-3, 3])
        ax.set_ylim([-3, 3])
        ax.set_zlim([-3, 3])
        plt.axis('off')
        
        frame_path = os.path.join(output_dir, f'frame_{i:03d}.png')
        plt.savefig(frame_path)
        plt.close()

generate_frames()

In [8]:
create_video()
from IPython.display import Video
Video('transformation_video.mp4')